In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [2]:
radii = 0.345
sdf_values = np.random.randn(1000, 1).astype(np.float32)
labels = radii > sdf_values

# If you don't scale the sigmoid, it won't recover the right threshold

In [3]:
tf.reset_default_graph()
s = tf.Session()

tf_labels = tf.placeholder(tf.float32, shape=(None, 1), name='labels')
tf_radius = tf.get_variable('radius', initializer=10.0)
tf_sdfs = tf.placeholder(tf.float32, shape=(None, 1), name='sdfs')
logits = 1*(tf_radius - tf_sdfs)
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=tf_labels))
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
gradients, variables = zip(*optimizer.compute_gradients(loss_op))
gradients, _ = tf.clip_by_global_norm(gradients, 5)
opt = optimizer.apply_gradients(zip(gradients, variables))
rgrad = tf.gradients(loss_op, tf_radius)

s.run(tf.global_variables_initializer())
loss = s.run([loss_op], feed_dict={tf_sdfs: sdf_values, tf_labels: labels})

for i in range(5000):
    loss, rad, rrg, _ = s.run([loss_op, tf_radius, rgrad, opt], feed_dict={tf_sdfs: sdf_values, tf_labels: labels})
    if abs(rad - radii) < 0.003:
        print("success!")
        break
print("done.")

Instructions for updating:
Colocations handled automatically by placer.
done.


# If you  scale the sigmoid, it works!

In [9]:
tf.reset_default_graph()
s = tf.Session()

tf_labels = tf.placeholder(tf.float32, shape=(None, 1), name='labels')
tf_radius = tf.get_variable('radius', initializer=10.0)
tf_sdfs = tf.placeholder(tf.float32, shape=(None, 1), name='sdfs')
logits = 100*(tf_radius - tf_sdfs)
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=tf_labels))
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
gradients, variables = zip(*optimizer.compute_gradients(loss_op))
gradients, _ = tf.clip_by_global_norm(gradients, 5)
opt = optimizer.apply_gradients(zip(gradients, variables))
rgrad = tf.gradients(loss_op, tf_radius)

s.run(tf.global_variables_initializer())
loss = s.run([loss_op], feed_dict={tf_sdfs: sdf_values, tf_labels: labels})

for i in range(5000):
    loss, rad, rrg, _ = s.run([loss_op, tf_radius, rgrad, opt], feed_dict={tf_sdfs: sdf_values, tf_labels: labels})
    if abs(rad - radii) < 0.003:
        print("success!")
        break
print("done.")

success!
done.
